# Feature Engineering
Process time: 20 seconds aprox. 

Traffic data use to be higly skewed, in this notebook we use quantile discretization for each category app to overcome that issue.

The result can be used to visually detect segments (e.g. music lovers) among other applications described in the next notebook.

- Note:
    - custom category apps can be analyzed by creating a new "category_apps_dictionary_file" and referencing it in to the ConfigFile.json file.
    - custom lists of potencial users can be analyzed  by creating a new "potential_high_value_users_file" and referencing it in to the ConfigFile.json file.
    


In [1]:
import os
import glob
import time
import json
import pandas as pd
import numpy as np

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Functions

## Target Users

In [4]:
def target_users_dict(json_file):
    """
    Return the dict of users.
    json_file(json): {"location_1":["user_1", "user_2", ... "user_n"],
                      "location_2":["user_1", "user_2", ... "user_n"] }
    """

    with open(json_file, "rb") as fp:
              data_dict = json.load(fp)
    return data_dict

## Wide Table functions

In [5]:
def pivot_apps(df, list_high_tdf_consumers):
    """
    Pivot apps to get a wide format.
    """

    columns = pd.Series(df.columns).str.strip()
    df.columns = columns
    df = df[pd.notnull(df['user_id'])]
    df= df.drop_duplicates()
    
    df = df[['user_id','app_id', 'traffic_bytes', 'duration_sec', 'frequency']].fillna(0.0)
    
    # filter only high TDF users
    mask_only_high_tdf_users = df['user_id'].isin(list_high_tdf_consumers) 
    df = df[mask_only_high_tdf_users]
    
    df = df.groupby(['user_id','app_id']).sum()
    df = df.reset_index().pivot(index='user_id', columns='app_id', values=['traffic_bytes', 'duration_sec', 'frequency'])
    df = df.fillna(0.0) # this line is very important, if you omit it you will lose columns, aggregations will fail!
    df.columns = df.columns.map(lambda x: '_'.join([str(i) for i in x]) )
    return df


## Get Wide Table

In [6]:
def get_wide_table(file_name, custom_categories_and_apps:dict, list_high_tdf_consumers):
    """
    Crete wide table including apps and categories.
    """
    dtypes = {
        'date': 'str',
        'user_id': 'Int64',
        'app_id': 'str',
        'time_label': 'str',
        'traffic_bytes': 'float64',
        'duration_sec': 'float64',
        'frequency': 'float64',
        'category_id': 'str',
        'english_name': 'str'}
    
    all_cols = [ 'date', 'user_id', 'app_id', 'time_label', 
                'traffic_bytes', 'duration_sec', 'frequency',
                'category_id', 'english_name']
    
    # Print info 
    all_apps_list = []
    for category, app_list, in custom_categories_and_apps.items():
        all_apps_list = all_apps_list + app_list
    print(f'Total categories: {len(custom_categories_and_apps.keys())}, Total apps: {len(all_apps_list)}')
    print("Categories: ", list(custom_categories_and_apps.keys()))
    
    # pivoting apps
    complete_pivoted_apps_df = pd.DataFrame()
    counter = 0
    chunksize = 10000000
    print(file_name) 
    df_chunk = pd.read_csv(file_name, 
                           encoding= 'utf-8',
                           sep=',',
                           chunksize=chunksize,
                           #names=columns_selected,
                           dtype=dtypes,
                           error_bad_lines=False,
                           warn_bad_lines=True,
                           #nrows=10000
                          )
    
    # Each chunck is in df format
    for chunk in df_chunk:
        
        temp_chunk = filter_apps_by_consultancy(df=chunk, app_list=all_apps_list)
        if temp_chunk.empty:
            print(f"Not data found in chunk for the: {category} category !!! shape({temp_chunk.shape})")
            print(f"Apps not found in chunk: {app_list}")
            continue
            
        temp_chunk = pivot_apps(df=temp_chunk, list_high_tdf_consumers=list_high_tdf_consumers)
        complete_pivoted_apps_df = pd.concat([complete_pivoted_apps_df, temp_chunk])
        complete_pivoted_apps_df = complete_pivoted_apps_df.groupby(['user_id']).agg('sum')
        complete_pivoted_apps_df = complete_pivoted_apps_df.fillna(0.0)
        
        print(f"DataFrame shape: {complete_pivoted_apps_df.shape} - chunk finished!")
        
    print()
    
    # horizontal sum of "apps_columns" to create "category_columns"
    
    print("Starting Horizontal sums ...")
    for category, app_list, in custom_categories_and_apps.items():
        # metric + app_id
        searchfor = app_list
        origin_cols = pd.Series(complete_pivoted_apps_df.columns)
        filtered_cols = origin_cols[origin_cols.str.contains('|'.join(searchfor))]
        #print (filtered_cols)
        
        # traffic mb
        filtered_cols_traffic = filtered_cols[filtered_cols.str.contains('traffic_bytes')]
        ###print (filtered_cols_traffic)
        complete_pivoted_apps_df[f'{category}_traffic_bytes'] = complete_pivoted_apps_df[filtered_cols_traffic].sum(axis=1, skipna=True)
        # duration sec
        filtered_cols_duration = filtered_cols[filtered_cols.str.contains('duration_sec')]
        complete_pivoted_apps_df[f'{category}_duration_sec'] = complete_pivoted_apps_df[filtered_cols_duration].sum(axis=1, skipna=True)
        ###print (filtered cols duration)
        
        # frequency
        filtered_cols_frequency = filtered_cols[filtered_cols.str.contains('frequency')]
        complete_pivoted_apps_df[f'{category}_frequency'] = complete_pivoted_apps_df[filtered_cols_frequency].sum(axis=1, skipna=True)
        ###print (filtered cols frequency)
        
        print(f'"{category}" - Wide Table created !')

    return complete_pivoted_apps_df

In [7]:
## TDF functions

## Quantile discretization

In [8]:
def jitter(a_series, noise_reduction=100000000):
    """
    Introduce a minimal amount of noise which artificially create unique bins edges.
    This avoid getting errors for non unique bin edges.
    """
    
    return (np.random.random(len(a_series))*a_series.std()/noise_reduction) - (a_series.std()/(2*noise_reduction))

In [9]:
def calculate_tdf_for_custom_categories(df, custom_categories_and_apps:dict, number_of_quantiles):
    """
    custom_categories_and_apps (dict): Format example:
    
    custom_categories_and_apps = {
        'BNC_classic_video' :[ 'YouTube','TikTok'],
        'BNC_Custom_video' : [ 'Netflix', 'AmazonInstantvideo', 'HBO', 'Disney' ] …
        }
    """
    dict_of_tdf_dataframes = {}
    all_columns = pd.Series(df.columns)
    for bnc_category, app in custom_categories_and_apps.items():
        filtered_cols = all_columns[all_columns.str.contains(bnc_category)]
        print(bnc_category)
        traffic_column_str = filtered_cols[filtered_cols.str.contains('traffic_bytes')].reset_index()[0][0]
        duration_column_str = filtered_cols[filtered_cols.str.contains('duration_sec')].reset_index()[0][0]
        frequency_column_str = filtered_cols[filtered_cols.str.contains('frequency')].reset_index()[0][0]
        
        # delete cells with zeros
        category_df = df.copy()
        category_df = category_df[filtered_cols]
        
        mask_zero_t = category_df[traffic_column_str] > 0.0
        mask_zero_d = category_df[duration_column_str] > 0.0
        mask_zero_f = category_df[frequency_column_str] > 0.0
        category_df = category_df[mask_zero_t & mask_zero_d & mask_zero_f]
        
        try:
            T_col = pd.qcut(x=category_df[traffic_column_str], q=number_of_quantiles, labels=range(1,number_of_quantiles+1))
            D_col = pd.qcut(x=category_df[duration_column_str], q=number_of_quantiles, labels=range(1,number_of_quantiles+1))
            F_col = pd.qcut(x=category_df[frequency_column_str], q=number_of_quantiles, labels=range(1,number_of_quantiles+1))
            print (f"{bnc_category} - TDF finished, no errors found.")
        except Exception as e:
            print(e)
            
            # Jitter correction method
            #T_col = pd.qcut(x=df[traffic_column_str] + jitter(df[traffic_column_str]), q=number_of_quantiles,labels=range(1,number_of_quantiles+1))
            #D_col = pd.qcut(x=df[duration_column_str] + jitter(df[duration_column_str]), q=number_of_quantiles, labels=range(1,number_of_quantiles+1))
            #F_col = pd.qcut(x=df[frequency_column_str] + jitter(df[frequency_column_str]), q=number_of_quantiles, labels=range(1,number_of_quantiles+1))
            #print("TDF calculation: Jitter function was applied !!!")
            #print()
            
            # Rank correction method
            T_col = pd.qcut(x=category_df[traffic_column_str].sort_values(ascending=False).rank(method='first'),
                            q=number_of_quantiles, 
                            labels=range(1,number_of_quantiles+1), 
                            duplicates='drop')
            D_col = pd.qcut(x=category_df[duration_column_str].sort_values(ascending=False).rank(method='first'), 
                            q=number_of_quantiles, 
                            labels=range(1,number_of_quantiles+1), 
                            duplicates='drop')
            F_col = pd.qcut(x=category_df[frequency_column_str].sort_values(ascending=False).rank(method='first'),
                            q=number_of_quantiles, 
                            labels=range(1,number_of_quantiles+1), 
                            duplicates='drop')
            print(f"{bnc_category} - TDF calculation: Rank function used for TDF calculation !!!")
            print()
            
            
            #T_col = pd.cut(x=df[traffic_column_str], bins=5, labels=range(1,number_of_quantiles+1))
            #D_col = pd.cut(x=df[duration_column_str], bins=5, labels=range(1,number_of_quantiles+1))
            #F_col = pd.cut(x=df[frequency_column_str], bins=5, labels=range(1,number_of_quantiles+1))
            #print ("=== Problem solved: "cut' used instead 'qcut' function used !!! ===")
            #print ()
            
        category_df['TScore_category'] = T_col
        category_df['DScore_category'] = D_col
        category_df['FScore_category'] = F_col
        
        category_df = category_df.sort_values(by=[traffic_column_str, duration_column_str, frequency_column_str], ascending=False)
        
        # be sure the order of columns is correct
        category_df.columns = [pd.Series(category_df.columns).str.contains('traffic_bytes').reset_index()[0][0],
                               pd.Series(category_df.columns).str.contains('duration_sec').reset_index()[0][0],
                               pd.Series(category_df.columns).str.contains('frequency').reset_index()[0][0],
                               pd.Series(category_df.columns).str.contains('TScore_category').reset_index()[0][0],
                               pd.Series(category_df.columns).str.contains('DScore_category').reset_index()[0][0],
                               pd.Series(category_df.columns).str.contains('FScore_category').reset_index()[0][0],
                              ]
        # rename columns
        category_df.columns = ['traffic_bytes', 'duration_sec', 'frequency', 'TScore_category', 'DScore_category', 'FScore_category', ]
        #category_df['city'] = city
        category_df['custom_category'] = bnc_category
        dict_of_tdf_dataframes[bnc_category] = category_df
    
    return dict_of_tdf_dataframes

In [10]:
def calculate_general_TDF(df, traffic_col, duration_col, frequency_col, number_of_quantiles):
    
    try:
        T_col = pd.qcut(x=df[traffic_col], q=number_of_quantiles, labels=range(1,number_of_quantiles+1))
        D_col = pd.qcut(x=df[duration_col], q=number_of_quantiles, labels=range(1,number_of_quantiles+1))
        F_col = pd.qcut(x=df[frequency_col], q=number_of_quantiles, labels=range(1,number_of_quantiles+1))
        print(f" TDF General finished, no errors found.")
    except Exception as e:
        print(e)
            #T_col = pd.qcut(x=df[traffic_column_str] + jitter(df[traffic_column_str]),q=number_of_quantiles, labels=range(1,number_of_quantiles+1))
            #D_col = pd.qcut(x=df[duration_column_str] + jitter(df[duration_column_str]), q=number_of_quantiles, labels=range(1,number_of_quantiles+1))
            #F_col = pd.qcut(x=df[frequency_column_str] + jitter(df[frequency_column_str]), q=number_of_quantiles, labels=range(1,number_of_quantiles+1))
            #print ("=== Problem solved: Jitter function was applied !!! ===") 
            
        T_col = pd.qcut(x=df[traffic_col].sort_values(ascending=False).rank(method='first'), q=number_of_quantiles, labels=range(1,number_of_quantiles+1))
        D_col = pd.qcut(x=df[duration_col].sort_values(ascending=False).rank(method='first'), q=number_of_quantiles, labels=range(1,number_of_quantiles+1))
        F_col = pd.qcut(x=df[frequency_col].sort_values(ascending=False).rank(method='first'), q=number_of_quantiles, labels= range(1,6))
        print(f"TDF calculation: Rank function used for TDF calculation !!! ")
        print()
        
        # T_col = pd.cut(x-df[traffic_column_str], bins=5, labels=range(1,number_of_quantiles+1))
        # D_col = pd.cut(x=df[duration_column_str], bins=5, labels=range(1,number_of_quantiles+1))
        # F_col = pd.cut(x=df[frequency_column_str], bins=5, labels=range(1,number_of_quantiles+1))
        # print ("=== Problem solved: 'cut' used instead 'gcut function used !!! ===")
        # print ()
        
    df['T'] = T_col
    df['D'] = D_col
    df['F'] = F_col
    
    df = df.sort_values(by=[traffic_col, duration_col, frequency_col], ascending=False)
    
    df['TDF_Score'] = df['T'].astype('str') + df['D'].astype('str') + df['F'].astype('str')
    df['TDF_Score'] = df['TDF_Score'].astype('int64')
    
    return df

## Concatenation

In [11]:
def concat_dict_of_dfs(dict_of_dfs):
    """
    concatenate all dataframes in the given dictionary of DataFrames.
    Notice that can be duplicates in "user_id" since two users can appear in more than one category.
    """
    class Unmatching(Exception):
        pass
    
    total_rows = 0
    for df in dict_of_dfs.values():
        total_rows = total_rows + len(df)
        
        
    list_of_dfs_to_concatenate = []
    for category in dict_of_dfs.keys():
        list_of_dfs_to_concatenate.append(dict_of_dfs[category].reset_index())
    concatenated_df = pd.concat(list_of_dfs_to_concatenate, axis=0, sort=False, ignore_index=True)

    if total_rows == concatenated_df.shape[0]:
        print("Concatenation succeeded!")
        return concatenated_df
    else:
        raise Unmatching( "Check the dataframes, total rows and rows in the concatenated dataframe don't match!")

## Other

In [12]:
def filter_apps_by_consultancy(df, app_list):
    mask= df['app_id'].isin(app_list)
    df=df[mask]
    return df

In [13]:
def wide_table_move_columns_to_the_end(df, primary_key, columns_list):
    """
    Reorder columns selected and put them at the end of the dataframe.
    This is just for the wide table.
    """
    columns_backup_df = df[columns_list]
    columns_list.remove(primary_key)
    df = df.drop(columns=columns_list)
    df = df.merge(columns_backup_df, how="inner", on=primary_key)
    return df

In [14]:
def delete_duplicated_user_in_dictionary(my_dict_of_users):
    """
    Get mutually exclusive dictionary of lists.
    This function deletes duplicates among groups and shows how many duplicates where found (which use to be not a big number).
    
    Input: my_dict of userss(dict): {
                                        'group 1': ['user_id_1', 'user_id_2' ... ],
                                        'group 2': ['user_id_1', 'user_id_2' ... ] }
    Example: One user that belongs to a group (lets say a city) cannot be in another group (lets say another city).
    
    >> test = {'a': [1,2,3,4,5],
               'b': [11,10,15,9],
               'c': [7,5,8,]}
            
    >> delete_duplicated_user_in_dictionary(test)
    
    >> WARNING! Duplicated users among groups.
    >> There are users that are in more than one group !!! 
    >> Duplicates found: (1)
    >> Deleting duplicated users
    >> {'a': [1, 2, 3, 4, 5], 'b': [11, 10, 15, 9], 'c': [7, 8]}
    """
    
    def are_duplicates(all_users):
        unique_users = list(set(all_users))
        if len(all_users) != len(unique_users):
            print("WARNING! Duplicated users among groups.")
            print ("There are users that are in more than one group !!! ")
            print (f"Duplicates found: ({len(all_users) - len(unique_users)})")
            print ("Deleting duplicated users")
        else:
            print("No duplicates found in dictionary of users !!!")
            
    # get a "unique List of users" (all_users_in_a_list)
    all_users_in_a_list = []
    for key, value in my_dict_of_users.items():
        all_users_in_a_list = all_users_in_a_list + value
    are_duplicates(all_users_in_a_list)
    all_users_in_a_list = pd.Series(all_users_in_a_list)
    all_users_in_a_list = all_users_in_a_list.drop_duplicates()
    all_users_in_a_list = list(all_users_in_a_list)
    
    # convert to pd.Series
    for key, value in my_dict_of_users.items():
        my_dict_of_users[key] = pd.Series(my_dict_of_users[key])
        
    # Get unique users and update "unique list of users"
    for key, value in my_dict_of_users.items():
        mask = my_dict_of_users[key].isin(all_users_in_a_list)
        my_dict_of_users[key] = my_dict_of_users[key][mask]
        all_users_in_a_list = pd.Series(all_users_in_a_list)
        all_users_in_a_list = all_users_in_a_list[~all_users_in_a_list.isin(my_dict_of_users[key])]
    
    # convert to list
    for key, value in my_dict_of_users.items():
        my_dict_of_users[key] = list(my_dict_of_users[key])
    
    return my_dict_of_users

In [15]:
def read_ConfigFile(file):
    import json
    with open(file, 'rb' ) as f:
        json_str = json.load(f)
    return json_str

# Main Program

In [16]:
ConfigFile = read_ConfigFile("ConfigFile.json")

In [17]:
print("SCRIPT 1 - Feature Engineering")
print()

print("ConfigFile Parameters: ")
for param, value in ConfigFile.items():
    print(param, " : ", value)

SCRIPT 1 - Feature Engineering

ConfigFile Parameters: 
innactive_time_between_app_session_in_seconds  :  60
sampling_period_in_hours  :  6
number_of_quantiles  :  9
potential_high_value_users_file  :  0_potential_high_value_users.json
category_apps_dictionary_file  :  all_category_apps_dictionary.json
country  :  China
final_report_folder_name  :  report_Dec2021


In [18]:
# ConfigFile parameters
innactive_time_between_app_session_in_seconds = ConfigFile['innactive_time_between_app_session_in_seconds']
sampling_period_in_hours = ConfigFile['sampling_period_in_hours']
number_of_quantiles = ConfigFile['number_of_quantiles']
potential_high_value_users_file = ConfigFile['potential_high_value_users_file']
category_apps_dictionary_file = ConfigFile['category_apps_dictionary_file']
country = ConfigFile['country']
final_report_folder_name = ConfigFile['final_report_folder_name']

In [19]:
# Load potential users
target_users_dict_ = target_users_dict(f"data/{potential_high_value_users_file}")
target_users_dict_ = delete_duplicated_user_in_dictionary(target_users_dict_)

No duplicates found in dictionary of users !!!


In [20]:
# Load apps to be analyzed
with open(f"data/1_category_apps_dictionaries/{category_apps_dictionary_file}", "rb") as fp:
    category_apps_dictionary = json.load(fp)

In [21]:
%%time
complete_wide_table = pd.DataFrame() # for apps and category apps
complete_stacked_table = pd.DataFrame() # for TDFs by category
for city, target_users_list in target_users_dict_.items():
    print(city)
    
    # wide table
    wide_table = get_wide_table(f"data/0_user_app_sampling_{sampling_period_in_hours}.csv",
                                custom_categories_and_apps=category_apps_dictionary,
                                list_high_tdf_consumers=target_users_dict_[city] )

    wide_table = wide_table.fillna(0.0)
    wide_table['country'] = country
    wide_table['city'] = city
    complete_wide_table = pd.concat([complete_wide_table, wide_table]).fillna(0.0)
    temp_cols = (complete_wide_table[["city", "country"]].reset_index()).drop_duplicates(subset='user_id').set_index("user_id")
    complete_wide_table = complete_wide_table.groupby(complete_wide_table.index).agg("sum")
    complete_wide_table = complete_wide_table.merge(temp_cols, on="user_id", how= "inner")
    
    # stacked table
    dict_TDF_category_apps = calculate_tdf_for_custom_categories(df=wide_table, 
                                                                 custom_categories_and_apps=category_apps_dictionary,
                                                                 number_of_quantiles=number_of_quantiles
                                                                )
    TDF_category_apps = concat_dict_of_dfs(dict_TDF_category_apps)
    TDF_category_apps['country'] = country
    TDF_category_apps['city'] = city
    complete_stacked_table = pd.concat([complete_stacked_table, TDF_category_apps])
    print(f"city{city} finished!")
    print()

big_chinese_city
Total categories: 20, Total apps: 2000
Categories:  ['Books', 'Business', 'Education', 'Entertainment', 'Finance', 'Games', 'Health&Fitness ', 'Lifestyle ', 'Music', 'Navigation', 'News', 'Photo&Video', 'References ', 'Shopping', 'Social_Networking', 'Travel', 'Utilities', 'Weather', 'infant&mom', 'sports']
data/0_user_app_sampling_6.csv
DataFrame shape: (871, 5088) - chunk finished!

Starting Horizontal sums ...
"Books" - Wide Table created !
"Business" - Wide Table created !
"Education" - Wide Table created !
"Entertainment" - Wide Table created !
"Finance" - Wide Table created !
"Games" - Wide Table created !
"Health&Fitness " - Wide Table created !
"Lifestyle " - Wide Table created !
"Music" - Wide Table created !
"Navigation" - Wide Table created !
"News" - Wide Table created !
"Photo&Video" - Wide Table created !
"References " - Wide Table created !
"Shopping" - Wide Table created !
"Social_Networking" - Wide Table created !
"Travel" - Wide Table created !
"Utili

In [22]:
# reorder columns wide table
temp_cols= pd.Series(complete_wide_table.columns)
searchfor= list(category_apps_dictionary.keys())
mask = temp_cols.str.contains('|'.join(searchfor))
temp_cols = list(temp_cols[mask])

complete_wide__table = complete_wide_table.reset_index()
complete_wide_table = wide_table_move_columns_to_the_end(complete_wide_table.reset_index(), primary_key="user_id", columns_list=["user_id"] + temp_cols)

complete_wide_table = wide_table_move_columns_to_the_end(df=complete_wide_table, primary_key="user_id", columns_list = ["user_id", "country", "city"])
complete_wide_table

,user_id,traffic_bytes_1,traffic_bytes_10,traffic_bytes_100,traffic_bytes_1000,traffic_bytes_1002,traffic_bytes_1003,traffic_bytes_1004,traffic_bytes_1005,traffic_bytes_1006,traffic_bytes_1007,traffic_bytes_1008,traffic_bytes_1010,traffic_bytes_1011,traffic_bytes_1012,traffic_bytes_1013,traffic_bytes_1014,traffic_bytes_1015,traffic_bytes_1016,traffic_bytes_1017,traffic_bytes_1018,traffic_bytes_1019,traffic_bytes_102,traffic_bytes_1020,traffic_bytes_1022,traffic_bytes_1023,traffic_bytes_1024,traffic_bytes_1025,traffic_bytes_1026,traffic_bytes_1028,traffic_bytes_1029,traffic_bytes_103,traffic_bytes_1030,traffic_bytes_1031,traffic_bytes_1032,traffic_bytes_1033,traffic_bytes_1034,traffic_bytes_1035,traffic_bytes_1036,traffic_bytes_1037,traffic_bytes_1038,traffic_bytes_1039,traffic_bytes_104,traffic_bytes_1040,traffic_bytes_1041,traffic_bytes_1042,traffic_bytes_1043,traffic_bytes_1044,traffic_bytes_1045,traffic_bytes_1047,traffic_bytes_1048,traffic_bytes_1049,traffic_bytes_105,traffic_bytes_1050,traffic_bytes_1052,traffic_bytes_1054,traffic_bytes_1055,traffic_bytes_1056,traffic_bytes_1057,traffic_bytes_1058,traffic_bytes_1059,traffic_bytes_1060,traffic_bytes_1061,traffic_bytes_1062,traffic_bytes_1064,traffic_bytes_1066,traffic_bytes_1067,traffic_bytes_1068,traffic_bytes_1069,traffic_bytes_107,traffic_bytes_1070,traffic_bytes_1073,traffic_bytes_1074,traffic_bytes_1075,traffic_bytes_1076,traffic_bytes_1077,traffic_bytes_1078,traffic_bytes_1079,traffic_bytes_108,traffic_bytes_1080,traffic_bytes_1081,traffic_bytes_1082,traffic_bytes_1083,traffic_bytes_1084,traffic_bytes_1086,traffic_bytes_1087,traffic_bytes_1088,traffic_bytes_1089,traffic_bytes_109,traffic_bytes_1090,traffic_bytes_1091,traffic_bytes_1092,traffic_bytes_1093,traffic_bytes_1094,traffic_bytes_1095,traffic_bytes_1096,traffic_bytes_1097,traffic_bytes_1098,traffic_bytes_1099,traffic_bytes_1100,traffic_bytes_1101,traffic_bytes_1103,traffic_bytes_1104,traffic_bytes_1105,traffic_bytes_1106,traffic_bytes_1107,traffic_bytes_1108,traffic_bytes_1109,traffic_bytes_111,traffic_bytes_1110,traffic_bytes_1111,traffic_bytes_1112,traffic_bytes_1113,traffic_bytes_1114,traffic_bytes_1116,traffic_bytes_1117,traffic_bytes_1118,traffic_bytes_1119,traffic_bytes_1120,traffic_bytes_1121,traffic_bytes_1122,traffic_bytes_1123,traffic_bytes_1124,traffic_bytes_1126,traffic_bytes_1127,traffic_bytes_1128,traffic_bytes_1129,traffic_bytes_113,traffic_bytes_1130,traffic_bytes_1132,traffic_bytes_1133,traffic_bytes_1134,traffic_bytes_1135,traffic_bytes_1136,traffic_bytes_1137,traffic_bytes_1138,traffic_bytes_1139,traffic_bytes_114,traffic_bytes_1140,traffic_bytes_1141,traffic_bytes_1142,traffic_bytes_1144,traffic_bytes_1145,traffic_bytes_1146,traffic_bytes_1149,traffic_bytes_115,traffic_bytes_1150,traffic_bytes_1151,traffic_bytes_1152,traffic_bytes_1154,traffic_bytes_1155,traffic_bytes_1156,traffic_bytes_1157,traffic_bytes_1158,traffic_bytes_1159,traffic_bytes_116,traffic_bytes_1160,traffic_bytes_1161,traffic_bytes_1162,traffic_bytes_1163,traffic_bytes_1164,traffic_bytes_1165,traffic_bytes_1166,traffic_bytes_1167,traffic_bytes_1169,traffic_bytes_1171,traffic_bytes_1172,traffic_bytes_1173,traffic_bytes_1174,traffic_bytes_1175,traffic_bytes_1178,traffic_bytes_1179,traffic_bytes_118,traffic_bytes_1180,traffic_bytes_1181,traffic_bytes_1183,traffic_bytes_1184,traffic_bytes_1186,traffic_bytes_1187,traffic_bytes_1188,traffic_bytes_1189,traffic_bytes_1190,traffic_bytes_1191,traffic_bytes_1192,traffic_bytes_1193,traffic_bytes_1194,traffic_bytes_1195,traffic_bytes_1196,traffic_bytes_1198,traffic_bytes_1199,traffic_bytes_12,traffic_bytes_120,traffic_bytes_1201,traffic_bytes_1202,traffic_bytes_1203,traffic_bytes_1204,traffic_bytes_1205,traffic_bytes_1206,traffic_bytes_1208,traffic_bytes_1209,traffic_bytes_121,traffic_bytes_1210,traffic_bytes_1211,traffic_bytes_1212,traffic_bytes_1214,traffic_bytes_1215,traffic_bytes_1216,traffic_bytes_1217,traffic_bytes_1218,traffic_bytes_1219,traffic_bytes_122,traffic_bytes_1220,traffi

In [23]:
complete_stacked_table

,user_id,traffic_bytes,duration_sec,frequency,TScore_category,DScore_category,FScore_category,custom_category,country,city
0,942,289313697.0,57409.0,2819.0,9,9,9,Books,China,big_chinese_city
1,836,126418688.0,8764.0,547.0,9,9,9,Books,China,big_chinese_city
2,772,89066952.0,10538.0,610.0,9,9,9,Books,China,big_chinese_city
3,793,80044876.0,3002.0,89.0,9,9,9,Books,China,big_chinese_city
4,655,78944081.0,5314.0,90.0,9,9,9,Books,China,big_chinese_city
...,...,...,...,...,...,...,...,...,...,...
10810,44,814.0,1.0,1.0,1,1,1,sports,China,big_chinese_city
10811,819,747.0,1.0,1.0,1,1,2,sports,China,big_chinese_city
10812,205,679.0,1.0,1.0,1,1,2,sports,China,big_chinese_city
10813,456,402.0,1.0,1.0,1,2,2,sports,China,big_chinese_city


In [24]:
# reorder columns stacked table
complete_stacked_table = complete_stacked_table[["user_id", 
                                                 "traffic_bytes",
                                                 "duration_sec",
                                                 "frequency",
                                                 "TScore_category",
                                                 "DScore_category",
                                                 "FScore_category",
                                                 "custom_category",
                                                 "country",
                                                 "city" ]]



In [25]:
complete_stacked_table

,user_id,traffic_bytes,duration_sec,frequency,TScore_category,DScore_category,FScore_category,custom_category,country,city
0,942,289313697.0,57409.0,2819.0,9,9,9,Books,China,big_chinese_city
1,836,126418688.0,8764.0,547.0,9,9,9,Books,China,big_chinese_city
2,772,89066952.0,10538.0,610.0,9,9,9,Books,China,big_chinese_city
3,793,80044876.0,3002.0,89.0,9,9,9,Books,China,big_chinese_city
4,655,78944081.0,5314.0,90.0,9,9,9,Books,China,big_chinese_city
...,...,...,...,...,...,...,...,...,...,...
10810,44,814.0,1.0,1.0,1,1,1,sports,China,big_chinese_city
10811,819,747.0,1.0,1.0,1,1,2,sports,China,big_chinese_city
10812,205,679.0,1.0,1.0,1,1,2,sports,China,big_chinese_city
10813,456,402.0,1.0,1.0,1,2,2,sports,China,big_chinese_city


In [26]:
complete_wide_table['user_id'].nunique() == complete_stacked_table['user_id'].nunique()

True

In [27]:
# round numbers
complete_wide_table= round(complete_wide_table, 6)
complete_wide_table

,user_id,traffic_bytes_1,traffic_bytes_10,traffic_bytes_100,traffic_bytes_1000,traffic_bytes_1002,traffic_bytes_1003,traffic_bytes_1004,traffic_bytes_1005,traffic_bytes_1006,traffic_bytes_1007,traffic_bytes_1008,traffic_bytes_1010,traffic_bytes_1011,traffic_bytes_1012,traffic_bytes_1013,traffic_bytes_1014,traffic_bytes_1015,traffic_bytes_1016,traffic_bytes_1017,traffic_bytes_1018,traffic_bytes_1019,traffic_bytes_102,traffic_bytes_1020,traffic_bytes_1022,traffic_bytes_1023,traffic_bytes_1024,traffic_bytes_1025,traffic_bytes_1026,traffic_bytes_1028,traffic_bytes_1029,traffic_bytes_103,traffic_bytes_1030,traffic_bytes_1031,traffic_bytes_1032,traffic_bytes_1033,traffic_bytes_1034,traffic_bytes_1035,traffic_bytes_1036,traffic_bytes_1037,traffic_bytes_1038,traffic_bytes_1039,traffic_bytes_104,traffic_bytes_1040,traffic_bytes_1041,traffic_bytes_1042,traffic_bytes_1043,traffic_bytes_1044,traffic_bytes_1045,traffic_bytes_1047,traffic_bytes_1048,traffic_bytes_1049,traffic_bytes_105,traffic_bytes_1050,traffic_bytes_1052,traffic_bytes_1054,traffic_bytes_1055,traffic_bytes_1056,traffic_bytes_1057,traffic_bytes_1058,traffic_bytes_1059,traffic_bytes_1060,traffic_bytes_1061,traffic_bytes_1062,traffic_bytes_1064,traffic_bytes_1066,traffic_bytes_1067,traffic_bytes_1068,traffic_bytes_1069,traffic_bytes_107,traffic_bytes_1070,traffic_bytes_1073,traffic_bytes_1074,traffic_bytes_1075,traffic_bytes_1076,traffic_bytes_1077,traffic_bytes_1078,traffic_bytes_1079,traffic_bytes_108,traffic_bytes_1080,traffic_bytes_1081,traffic_bytes_1082,traffic_bytes_1083,traffic_bytes_1084,traffic_bytes_1086,traffic_bytes_1087,traffic_bytes_1088,traffic_bytes_1089,traffic_bytes_109,traffic_bytes_1090,traffic_bytes_1091,traffic_bytes_1092,traffic_bytes_1093,traffic_bytes_1094,traffic_bytes_1095,traffic_bytes_1096,traffic_bytes_1097,traffic_bytes_1098,traffic_bytes_1099,traffic_bytes_1100,traffic_bytes_1101,traffic_bytes_1103,traffic_bytes_1104,traffic_bytes_1105,traffic_bytes_1106,traffic_bytes_1107,traffic_bytes_1108,traffic_bytes_1109,traffic_bytes_111,traffic_bytes_1110,traffic_bytes_1111,traffic_bytes_1112,traffic_bytes_1113,traffic_bytes_1114,traffic_bytes_1116,traffic_bytes_1117,traffic_bytes_1118,traffic_bytes_1119,traffic_bytes_1120,traffic_bytes_1121,traffic_bytes_1122,traffic_bytes_1123,traffic_bytes_1124,traffic_bytes_1126,traffic_bytes_1127,traffic_bytes_1128,traffic_bytes_1129,traffic_bytes_113,traffic_bytes_1130,traffic_bytes_1132,traffic_bytes_1133,traffic_bytes_1134,traffic_bytes_1135,traffic_bytes_1136,traffic_bytes_1137,traffic_bytes_1138,traffic_bytes_1139,traffic_bytes_114,traffic_bytes_1140,traffic_bytes_1141,traffic_bytes_1142,traffic_bytes_1144,traffic_bytes_1145,traffic_bytes_1146,traffic_bytes_1149,traffic_bytes_115,traffic_bytes_1150,traffic_bytes_1151,traffic_bytes_1152,traffic_bytes_1154,traffic_bytes_1155,traffic_bytes_1156,traffic_bytes_1157,traffic_bytes_1158,traffic_bytes_1159,traffic_bytes_116,traffic_bytes_1160,traffic_bytes_1161,traffic_bytes_1162,traffic_bytes_1163,traffic_bytes_1164,traffic_bytes_1165,traffic_bytes_1166,traffic_bytes_1167,traffic_bytes_1169,traffic_bytes_1171,traffic_bytes_1172,traffic_bytes_1173,traffic_bytes_1174,traffic_bytes_1175,traffic_bytes_1178,traffic_bytes_1179,traffic_bytes_118,traffic_bytes_1180,traffic_bytes_1181,traffic_bytes_1183,traffic_bytes_1184,traffic_bytes_1186,traffic_bytes_1187,traffic_bytes_1188,traffic_bytes_1189,traffic_bytes_1190,traffic_bytes_1191,traffic_bytes_1192,traffic_bytes_1193,traffic_bytes_1194,traffic_bytes_1195,traffic_bytes_1196,traffic_bytes_1198,traffic_bytes_1199,traffic_bytes_12,traffic_bytes_120,traffic_bytes_1201,traffic_bytes_1202,traffic_bytes_1203,traffic_bytes_1204,traffic_bytes_1205,traffic_bytes_1206,traffic_bytes_1208,traffic_bytes_1209,traffic_bytes_121,traffic_bytes_1210,traffic_bytes_1211,traffic_bytes_1212,traffic_bytes_1214,traffic_bytes_1215,traffic_bytes_1216,traffic_bytes_1217,traffic_bytes_1218,traffic_bytes_1219,traffic_bytes_122,traffic_bytes_1220,traffi

In [28]:
# round numbers (except "user_id")
exclude_columns = ['user_id', 'Score_category', 'Dscore_category', 'Fscore_category', 'custom_category', 'country', 'city']
complete_stacked_table.loc[:, ~complete_stacked_table.columns.isin(exclude_columns)]= round(complete_stacked_table.loc[:, ~complete_stacked_table.columns.isin(exclude_columns)], 6)
complete_stacked_table

,user_id,traffic_bytes,duration_sec,frequency,TScore_category,DScore_category,FScore_category,custom_category,country,city
0,942,289313697.0,57409.0,2819.0,9,9,9,Books,China,big_chinese_city
1,836,126418688.0,8764.0,547.0,9,9,9,Books,China,big_chinese_city
2,772,89066952.0,10538.0,610.0,9,9,9,Books,China,big_chinese_city
3,793,80044876.0,3002.0,89.0,9,9,9,Books,China,big_chinese_city
4,655,78944081.0,5314.0,90.0,9,9,9,Books,China,big_chinese_city
...,...,...,...,...,...,...,...,...,...,...
10810,44,814.0,1.0,1.0,1,1,1,sports,China,big_chinese_city
10811,819,747.0,1.0,1.0,1,1,2,sports,China,big_chinese_city
10812,205,679.0,1.0,1.0,1,1,2,sports,China,big_chinese_city
10813,456,402.0,1.0,1.0,1,2,2,sports,China,big_chinese_city


# Feture engineering
Quantile discretization by app category.


Get a wide table with cells filled of Quantile Discretization labels by category app.


In [29]:
complete_stacked_table['TDF_Score_By_Custom_Category'] = complete_stacked_table['TScore_category'].astype('str') + complete_stacked_table['DScore_category'].astype('str') + complete_stacked_table['FScore_category'].astype('str')

In [30]:
# Explore users
complete_stacked_table.sort_values(by=['user_id']).head(10)

,user_id,traffic_bytes,duration_sec,frequency,TScore_category,DScore_category,FScore_category,custom_category,country,city,TDF_Score_By_Custom_Category
7142,0,395617.0,923.0,64.0,6,8,8,Shopping,China,big_chinese_city,688
567,0,503855.0,26.0,2.0,6,3,3,Education,China,big_chinese_city,633
10711,0,4635.0,5.0,3.0,3,4,4,sports,China,big_chinese_city,344
908,0,913092.0,1351.0,109.0,6,7,8,Entertainment,China,big_chinese_city,678
2181,0,6680540.0,3170.0,173.0,7,7,7,Games,China,big_chinese_city,777
7819,0,36300392.0,18693.0,999.0,7,8,8,Social_Networking,China,big_chinese_city,788
106,0,645288.0,42.0,19.0,8,5,7,Books,China,big_chinese_city,857
1523,0,657304.0,504.0,54.0,8,7,8,Finance,China,big_chinese_city,878
3822,0,5470238.0,3896.0,151.0,8,9,8,Music,China,big_chinese_city,898
6745,0,42884.0,88.0,9.0,4,6,6,References,China,big_chinese_city,466


In [31]:
pivoted_complete_stacked_table = complete_stacked_table.pivot(index='user_id', columns='custom_category', values='TDF_Score_By_Custom_Category')
pivoted_complete_stacked_table = pivoted_complete_stacked_table.fillna(0)
pivoted_complete_stacked_table

custom_category,Books,Business,Education,Entertainment,Finance,Games,Health&Fitness,Lifestyle,Music,Navigation,News,Photo&Video,References,Shopping,Social_Networking,Travel,Utilities,Weather,infant&mom,sports
user_id,,,,,,,,,,,,,,,,,,,,
0,857,0,633,678,878,777,0,999,898,788,656,0,466,688,788,899,788,346,0,344
1,0,0,0,442,0,334,0,242,665,243,211,0,111,321,653,121,333,0,0,0
2,354,0,0,353,0,445,0,123,786,222,233,0,0,0,865,566,454,0,0,861
3,221,0,899,344,987,676,112,898,778,776,556,112,233,776,667,453,777,778,0,455
4,0,0,0,756,724,623,0,734,778,423,445,646,0,445,655,322,555,543,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,778,0,321,867,976,877,0,988,889,778,668,989,678,677,889,777,878,856,0,788
996,571,0,464,564,465,445,0,545,576,334,545,223,566,767,766,345,233,888,0,446
997,211,0,0,556,776,766,0,987,888,876,676,442,223,743,666,432,775,999,0,0


In [32]:
general_tdf = complete_stacked_table[["user_id", "traffic_bytes", "duration_sec", "frequency"]].groupby("user_id").sum()

In [33]:
general_tdf = calculate_general_TDF(general_tdf, 
                                    traffic_col= 'traffic_bytes',
                                    duration_col= 'duration_sec', 
                                    frequency_col = 'frequency',
                                    number_of_quantiles=number_of_quantiles
                                   )

 TDF General finished, no errors found.


In [34]:
general_tdf = round(general_tdf, 6)

In [35]:
general_tdf

,traffic_bytes,duration_sec,frequency,T,D,F,TDF_Score
user_id,,,,,,,
942,3.743837e+10,12523302.0,474051.0,9,9,9,999
772,8.505699e+09,1893671.0,82460.0,9,9,9,999
837,2.954388e+09,843211.0,44828.0,9,9,9,999
723,2.199940e+09,718885.0,31017.0,9,9,9,999
667,1.974879e+09,447621.0,22576.0,9,9,9,999
...,...,...,...,...,...,...,...
578,8.754400e+04,373.0,31.0,1,1,1,111
468,8.129800e+04,20.0,16.0,1,1,1,111
252,2.192800e+04,18.0,10.0,1,1,1,111


In [36]:
final_TDF_table = general_tdf.merge(pivoted_complete_stacked_table, on='user_id', how='inner')
final_TDF_table = final_TDF_table.merge(complete_wide_table[['user_id', 'country', 'city']], on='user_id', how='inner')

In [37]:
final_TDF_table

,user_id,traffic_bytes,duration_sec,frequency,T,D,F,TDF_Score,Books,Business,Education,Entertainment,Finance,Games,Health&Fitness,Lifestyle,Music,Navigation,News,Photo&Video,References,Shopping,Social_Networking,Travel,Utilities,Weather,infant&mom,sports,country,city
0,942,3.743837e+10,12523302.0,474051.0,9,9,9,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,China,big_chinese_city
1,772,8.505699e+09,1893671.0,82460.0,9,9,9,999,999,0,999,999,999,999,787,999,999,999,999,989,999,999,999,999,999,999,961,999,China,big_chinese_city
2,837,2.954388e+09,843211.0,44828.0,9,9,9,999,999,0,987,999,999,999,345,999,999,999,999,999,999,999,999,999,999,999,0,999,China,big_chinese_city
3,723,2.199940e+09,718885.0,31017.0,9,9,9,999,899,0,442,999,999,999,0,999,999,999,999,999,999,999,999,999,999,999,431,999,China,big_chinese_city
4,667,1.974879e+09,447621.0,22576.0,9,9,9,999,899,0,998,999,999,999,486,999,999,999,999,567,999,999,999,999,999,999,0,889,China,big_chinese_city
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
866,578,8.754400e+04,373.0,31.0,1,1,1,111,0,0,0,111,0,111,0,121,0,111,111,0,0,0,111,0,111,0,0,0,China,big_chinese_city
867,468,8.129800e+04,20.0,16.0,1,1,1,111,0,0,0,0,0,111,0,111,0,111,0,0,0,0,111,0,111,0,0,0,China,big_chinese_city
868,252,2.192800e+04,18.0,10.0,1,1,1,111,0,0,0,0,0,0,0,111,0,111,111,0,0,0,111,0,111,0,0,0,China,big_chinese_city
869,295,2.086200e+04,22.0,13.0,1,1,1,111,0,0,0,0,0,0,0,0,0,111,0,0,0,0,111,0,111,0,0,0,China,big_chinese_city


# Save files

In [38]:
create_new_directory = f'data/2_reports/S{sampling_period_in_hours}Q{number_of_quantiles}_{final_report_folder_name}/'
if not os.path.exists(create_new_directory):
    os.makedirs(create_new_directory)

In [39]:
print( "Output files allocation: ", create_new_directory)

Output files allocation:  data/2_reports/S6Q9_report_Dec2021/


In [40]:
complete_wide_table.to_csv(create_new_directory + f'APPS_table_wide_format.csv',
                           sep=',',
                           encoding='utf-8',
                           index=False)

In [41]:
complete_stacked_table.to_csv(create_new_directory + f'QUANTILIZED_table_stacked_format_S{sampling_period_in_hours}Q{number_of_quantiles}.csv',
                              sep=',',
                              encoding='utf-8',
                              index=False)

In [42]:
final_TDF_table.to_csv(create_new_directory + f'QUANTILIZED_table_wide_format_S{sampling_period_in_hours}Q{number_of_quantiles}.csv',
                       sep=',',
                       encoding= 'utf-8',
                       index=False)
